In [63]:
import numpy as np
import multiprocessing as mp
import pickle as pk
#import cvxpy as cp


ModuleNotFoundError: No module named 'cvxpy'

In [62]:


x = np.ones(10)
manager = mp.Manager()

dataDict=manager.dict()
dataDict['x']=x
print(dataDict)

print(x)

dataDict['x'][0]=5
print(dataDict['x'])
print(x)

print("Action")

x = np.ones(1)
shm = mp.shared_memory.SharedMemory(create=True, size=1000)

shm.buf[:len(b'1234')]=b'1234'
print(b'1234')

val = b'1234'
shm.buf[:len(val)]=val

print("worked")


val = pk.dumps(x)
shm.buf[:len(val)]=val

print("worked2")

print(shm.buf[:])

#shm.buf[:4]=bytearray([1,2,3,4])
#print(len(pk.dumps(x)))
#print(memoryview(x)[:]) #bytearray(x)
#print(gla)


{'x': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Action
b'1234'
worked
worked2


In [ ]:


#shm.buf=
#print(shm.buf.append)

#a[0][0]=1
print(a[0])
print(x)


# # Parallel processing
# def my_func(i, d):
#     d['x']=x
#     x[i] = 0
#     d['x']=x
#     #print(d['x'],i)
#     #d[str(i)]=i


# pool = mp.Pool(processes=4)
# [pool.apply(my_func,args=[i,dataDict,]) for i in range(4)]

# print(x)
# print(dataDict['x'])

In [65]:
from multiprocessing import Process, Manager
from multiprocessing.managers import BaseManager

class SimpleClass(object):
    def __init__(self):
        self.var = 0

    def set(self, value):
        self.var = value

    def get(self):
        return self.var


def change_obj_value(obj):
    obj.set(100)


if __name__ == '__main__':
    BaseManager.register('SimpleClass', SimpleClass)
    manager = BaseManager()
    manager.start()
    inst = manager.SimpleClass()

    p = Process(target=change_obj_value, args=[inst])
    p.start()
    p.join()

    print (inst)                    # <__main__.SimpleClass object at 0x10cf82350>
    print( inst.get())              # 100

100


In [131]:
from multiprocessing import Process, Manager
from multiprocessing.managers import BaseManager

class SharedClass(object):
    def __init__(self):
        self.var = [0,1]
        self.constr = [0]

    def setXloc(self,loc,val):
        self.var[loc]=val
        
    def setX(self, value):
        self.var = value

    def getX(self):
        return [self.var]
    
    def appConstr(self, value):
        self.constr += value
        
    def getConstr(self):
        return self.constr

def change_obj_value(obj):
    #obj.setX(100)
    X=obj.getX()
    #X=obj.var
    obj.appConstr(X)    
    


if __name__ == '__main__':
    BaseManager.register('SharedClass', SharedClass)
    manager = BaseManager()
    manager.start()
    shr = manager.SharedClass()

    p = Process(target=change_obj_value, args=[shr])
    p.start()
    p.join()

    print (shr)                    # <__main__.SimpleClass object at 0x10cf82350>
    print( shr.getX() )              # 100
    print(shr.getConstr())

[[0, 1]]
[0, [0, 1]]


In [134]:
shr.setXloc(0,1)

In [135]:
print(shr.getConstr())

[0, [0, 1]]


In [98]:
print(shr.getX())

[1, 1]


# Testing a basic idea

In [86]:
X=[1,2]
constr=[X]

In [89]:
print(constr)

[[2, 2]]


In [88]:
X[0]=2

In [106]:
print(constr)

[[2, 2]]


# The Sharing Based Method doesn't pass instances

In [140]:
class sharedX:
    def __init__(s):
        s.X=[0,1]
        s.constr=[-1]
    
    def getX(s):
        return [s.X]
    
    def setXloc(s,loc,val):
        s.X[loc]=val
        
    
    def getConstr(s):
        return s.constr
    
    def appendConstr(s,val):
        try:
            s.constr += val
        except:
            s.constr += [val]

sX = sharedX()

sX.appendConstr(sX.getX())
#sX.appendConstr([sX.X]) #sX.getX())
#sX.constr += [sX.X]


print(sX.getConstr())

#sX.X[0]=1
sX.setXloc(0,1)

print(sX.getConstr())


[-1, [0, 1]]
[-1, [1, 1]]


In [141]:
class sharedX:
    def __init__(s):
        s.X=[0,1]
        s.constr=[-1]
    
    def getX(s):
        return [s.X]
    
    def setXloc(s,loc,val):
        s.X[loc]=val
        
    
    def getConstr(s):
        return s.constr
    
    def appendConstr(s,val):
        try:
            s.constr += val
        except:
            s.constr += [val]


def worker(sX):
    sX.appendConstr(sX.getX())
    print(sX.getConstr())
    #sX.X[0]=1
    sX.setXloc(0,1)
    print(sX.getConstr())


BaseManager.register('sharedX', sharedX)
manager = BaseManager()
manager.start()
sX = manager.sharedX()

p = Process(target=worker, args=[sX])
p.start()
p.join()



[-1, [0, 1]]
[-1, [0, 1]]


In [143]:
import ctypes

a = "hello world"
print(ctypes.cast(id(a), ctypes.py_object).value)


hello world
